In [1]:
from datasets.CSVDataset import CSVDataset
from utils import reduce_dataset
from train import kfold_grid_search
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = CSVDataset("data/features/Resnet18Model_features.csv")
dataset = reduce_dataset(dataset, discard_ratio=0.0)
print(f"{len(dataset):,} examples")
kfold_grid_search(dataset, in_features=512, device=device,
                  checkpoint_file_path=None,  #"grid-search-25.json",
                  batch_size=256,
                  max_epochs=20,
                  hidden_layer_combs=[1,2,3],
                  unit_combs=[512,1024,2048],
                  learning_rate_combs=[0.0001],
                  dropout_combs=[0.2,0.3,0.4],
                  threshold_combs=[0.5],
                  focal_alpha_combs=[0.02, 0.03, 0.04],
                  )


data/features/Resnet18Model_features.pickle
20,227 examples
(0/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.02, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 512.56it/s]


{'test_loss': 0.0029077863960992544, 'test_accuracy': 0.6771925759050141, 'test_precision': 0.13090249196949907, 'test_recall': 0.678352520391321, 'test_f1': 0.2080584550744244, 'test_mcc': 0.1897851169839782}
(1/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.03, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 1026.10it/s]


{'test_loss': 0.004358344367938116, 'test_accuracy': 0.6869263209840457, 'test_precision': 0.14146383785703467, 'test_recall': 0.6514056118315398, 'test_f1': 0.20991091300613998, 'test_mcc': 0.1893924281019141}
(2/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.04, focal_gamma=2.0)


...

{'test_loss': 0.0031561152631184087, 'test_accuracy': 0.6719635219334597, 'test_precision': 0.1477642620033271, 'test_recall': 0.6948931309273801, 'test_f1': 0.21749905533023234, 'test_mcc': 0.2034162093315004}
(79/81) (hidden_layers=3, units=2048, dropout=0.4, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.03, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 512.54it/s]


{'test_loss': 0.005201930747716688, 'test_accuracy': 0.650630880649864, 'test_precision': 0.12846721858240484, 'test_recall': 0.691379715807461, 'test_f1': 0.20341440360429167, 'test_mcc': 0.18480070787650346}
(80/81) (hidden_layers=3, units=2048, dropout=0.4, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.04, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 512.04it/s]

{'test_loss': 0.0075445526366820555, 'test_accuracy': 0.6107455012912918, 'test_precision': 0.11354037516269178, 'test_recall': 0.7536908428160994, 'test_f1': 0.1897553554056984, 'test_mcc': 0.17940185170737538}

Best params: (hidden_layers=1, units=2048, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.04, focal_gamma=2.0)
Best performance: {'test_loss': 0.0043621350836474445, 'test_accuracy': 0.7990874771526462, 'test_precision': 0.18904178281316106, 'test_recall': 0.7057340472457538, 'test_f1': 0.2901641352772048, 'test_mcc': 0.2886457161334991}
